# Building a Collaborative Filtering Recommender System with implict feedback

Collaborative Filtering (CF) is a method of making automatic predictions about the interests of a user by learning its preferences (or taste) based on information of his engagements with a set of available items, along with other users’ engagements with the same set of items. in other words, CF assumes that, if a person A has the same opinion as person B on some set of issues X={x1,x2,…}, then A is more likely to have B‘s opinion on a new issue y than to have the opinion of any other person that doesn’t agree with A on X. It is a technique that can filter out items that a user might like on the basis of reaction by similar useres it works by seaching a large group of people and finding a smaller set of users with tastes similar to a particular user

Explicit vs Implicit

Let’s face it, explicit feedback data is hard to collect as they require additional input from the users. The users give explicit feedback only when they choose to do so. As a result, most of the time, people don’t provide ratings at all (I myself totally guilty of this on Konga or Jumia popular e-commerce site in Nigeria). Therefore, the amount of explicit data collected are extremely scarce.

On the other hand, implicit data is easy to collect in large quantities without any effort from the users. The goal is to convert user behavior into user preferences which indirectly reflect opinion through observing user behavior. For example, a user that bookmarked many articles by the same author probably likes that author.

In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
from pandas.api.types import CategoricalDtype
import pyodbc

In [2]:
# connect to SQL (1st step)

# establish an open connection to SQL
conn = pyodbc.connect('Driver={SQL Server};'
'Server=DESKTOP-1VJ4H95\MSSQLSERVER01;'
'Database=AdventureWorksDW2017;'
'Trusted_Connection=yes;')

In [3]:
 # now that the database is connected to python pull the data needed (2nd step)

# pull orderdate Region and revenue  from SQL (AdventureWorks Database)
# plug your SQL query inside the """ """
retail_data=pd.read_sql_query("""
 SELECT 

P.ProductKey AS Product_ID
,P.EnglishProductName AS Item_Description
,FIS.CustomerKey AS Customer_ID
,FIS.OrderQuantity
--,FIS.SalesOrderNumber
 ,FIS.SalesAmount AS Revenue
 ,FIS.TotalProductCost AS Cost 
FROM [dbo].[DimProduct] P

INNER JOIN [dbo].[FactInternetSales] FIS
ON P.ProductKey = FIS.ProductKey
""", conn)

conn.close() # please close it after 


In [4]:
retail_data.head()

,Product_ID,Item_Description,Customer_ID,OrderQuantity,Revenue,Cost
0,310,"Road-150 Red, 62",21768,1,3578.2700,2171.2942
1,346,"Mountain-100 Silver, 44",28389,1,3399.9900,1912.1544
2,346,"Mountain-100 Silver, 44",25863,1,3399.9900,1912.1544
3,336,"Road-650 Black, 62",14501,1,699.0982,413.1463
4,346,"Mountain-100 Silver, 44",11003,1,3399.9900,1912.1544


In [5]:
retail_data.shape

(60398, 6)

In [6]:
#Let’s check to see if there are any missing values in the data.

retail_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60398 entries, 0 to 60397
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Product_ID        60398 non-null  int64  
 1   Item_Description  60398 non-null  object 
 2   Customer_ID       60398 non-null  int64  
 3   OrderQuantity     60398 non-null  int64  
 4   Revenue           60398 non-null  float64
 5   Cost              60398 non-null  float64
dtypes: float64(2), int64(3), object(1)
memory usage: 2.8+ MB


In [7]:
# now let us change the data type of ProductID data type to object
retail_data.Product_ID = retail_data.Product_ID.astype(str)

#Let’s check to see if thechange has been implemented
retail_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60398 entries, 0 to 60397
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Product_ID        60398 non-null  object 
 1   Item_Description  60398 non-null  object 
 2   Customer_ID       60398 non-null  int64  
 3   OrderQuantity     60398 non-null  int64  
 4   Revenue           60398 non-null  float64
 5   Cost              60398 non-null  float64
dtypes: float64(2), int64(2), object(2)
memory usage: 2.8+ MB


In [8]:
retail_data.isnull().sum()

Product_ID          0
Item_Description    0
Customer_ID         0
OrderQuantity       0
Revenue             0
Cost                0
dtype: int64

##### Most columns have no missing values

In [9]:
# since there is no missing values,let me just rename the data set to 'cleaned_retail'
cleaned_retail = retail_data

In [10]:
cleaned_retail.isnull().sum()

Product_ID          0
Item_Description    0
Customer_ID         0
OrderQuantity       0
Revenue             0
Cost                0
dtype: int64

clearly, we have no missing values and all of the purchases can be matched to a specific customer.
Before we make any sort of ratings matrix, it would be nice to have a lookup table that keeps track of each item ID along with a description of that item. Let’s make that now.

In [11]:
item_lookup = cleaned_retail[['Product_ID', 'Item_Description']].drop_duplicates() # Only get unique item/description pairs
item_lookup['Product_ID'] = item_lookup.Product_ID.astype(str) # Encode as strings for future lookup ease

In [12]:
item_lookup.head()

,Product_ID,Item_Description
0,310,"Road-150 Red, 62"
1,346,"Mountain-100 Silver, 44"
3,336,"Road-650 Black, 62"
5,311,"Road-150 Red, 44"
7,351,"Mountain-100 Black, 48"


#### This can tell us what each item is, such as that product_ID 310 is a Road-150 Red, 62. Now that this has been created, we need to:

1. Group purchase quantities together by Product_ID and item ID
2. Change any sums that equal zero to one (this can happen if items were returned, but we want to indicate that the user actually purchased the item instead of assuming no interaction between the user and the item ever took place)
3. Only include customers with a positive purchase total to eliminate possible errors
4. Set up our sparse ratings matrix

In [13]:
cleaned_retail['Customer_ID'] = cleaned_retail.Customer_ID.astype(int) # Convert to int for customer ID
cleaned_retail = cleaned_retail[['Product_ID', 'OrderQuantity', 'Customer_ID']] # Get rid of unnecessary info
grouped_cleaned = cleaned_retail.groupby(['Customer_ID', 'Product_ID']).sum().reset_index() # Group together
grouped_cleaned.OrderQuantity.loc[grouped_cleaned.OrderQuantity == 0] = 1 # Replace a sum of zero purchases with a one to
# indicate purchased
grouped_purchased = grouped_cleaned.query('OrderQuantity > 0') # Only get customers where purchase totals were positive

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [14]:
grouped_purchased.head()

,Customer_ID,Product_ID,OrderQuantity
0,11000,214,1
1,11000,344,1
2,11000,353,1
3,11000,485,1
4,11000,488,1


#### Instead of representing an explicit rating, the purchase quantity can represent a “confidence” in terms of how strong the interaction was. Items with a larger number of purchases by a customer can carry more weight in our ratings matrix of purchases.

Our last step is to create the sparse ratings matrix of users and items utilizing the code below:

In [15]:
customers = list(np.sort(grouped_purchased.Customer_ID.unique())) # Get our unique customers
products = list(grouped_purchased.Product_ID.unique()) # Get our unique products that were purchased
quantity = list(grouped_purchased.OrderQuantity) # All of our purchases

rows = grouped_purchased.Customer_ID.astype(CategoricalDtype( categories = customers)).cat.codes 
# Get the associated row indices
cols = grouped_purchased.Product_ID.astype(CategoricalDtype( categories = products)).cat.codes 
# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

In [16]:
customers[:5]

[11000, 11001, 11002, 11003, 11004]

In [17]:
purchases_sparse

<18484x158 sparse matrix of type '<class 'numpy.intc'>'
	with 59051 stored elements in Compressed Sparse Row format>

#### We have 18,484 customers with 158 items. For these user/item interactions, 59051 of these items had a purchase. In terms of sparsity of the matrix, that makes:

In [18]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

97.97803231806365

98% of the interaction matrix is sparse. For collaborative filtering to work, the maximum sparsity you could get away with would probably be about 99.5% or so. We are well below this, so we should be able to get decent results.

### Creating a Training and Validation Set

In [19]:
import random

In [20]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

In [21]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

### This will return our training set, a test set that has been binarized to 0/1 for purchased/not purchased, and a list of which users had at least one item masked. We will test the performance of the recommender system on these users only. I am masking 20% of the user/item interactions for this example.

#### Implementing ALS for Implicit Feedback

In [22]:
def implicit_weighted_ALS(training_set, lambda_val = 0.1, alpha = 40, iterations = 10, rank_size = 20, seed = 0):
    '''
    Implicit weighted ALS taken from Hu, Koren, and Volinsky 2008. Designed for alternating least squares and implicit
    feedback based collaborative filtering. 
    
    parameters:
    
    training_set - Our matrix of ratings with shape m x n, where m is the number of users and n is the number of items.
    Should be a sparse csr matrix to save space. 
    
    lambda_val - Used for regularization during alternating least squares. Increasing this value may increase bias
    but decrease variance. Default is 0.1. 
    
    alpha - The parameter associated with the confidence matrix discussed in the paper, where Cui = 1 + alpha*Rui. 
    The paper found a default of 40 most effective. Decreasing this will decrease the variability in confidence between
    various ratings.
    
    iterations - The number of times to alternate between both user feature vector and item feature vector in
    alternating least squares. More iterations will allow better convergence at the cost of increased computation. 
    The authors found 10 iterations was sufficient, but more may be required to converge. 
    
    rank_size - The number of latent features in the user/item feature vectors. The paper recommends varying this 
    between 20-200. Increasing the number of features may overfit but could reduce bias. 
    
    seed - Set the seed for reproducible results
    
    returns:
    
    The feature vectors for users and items. The dot product of these feature vectors should give you the expected 
    "rating" at each point in your original matrix. 
    '''
    
    # first set up our confidence matrix
    
    conf = (alpha*training_set) # To allow the matrix to stay sparse, I will add one later when each row is taken 
                                # and converted to dense. 
    num_user = conf.shape[0]
    num_item = conf.shape[1] # Get the size of our original ratings matrix, m x n
    
    # initialize our X/Y feature vectors randomly with a set seed
    rstate = np.random.RandomState(seed)
    
    X = sparse.csr_matrix(rstate.normal(size = (num_user, rank_size))) # Random numbers in a m x rank shape
    Y = sparse.csr_matrix(rstate.normal(size = (num_item, rank_size))) # Normally this would be rank x n but we can 
                                                                 # transpose at the end. Makes calculation more simple.
    X_eye = sparse.eye(num_user)
    Y_eye = sparse.eye(num_item)
    lambda_eye = lambda_val * sparse.eye(rank_size) # Our regularization term lambda*I. 
    
    # We can compute this before iteration starts. 
    
    # Begin iterations
   
    for iter_step in range(iterations): # Iterate back and forth between solving X given fixed Y and vice versa
        # Compute yTy and xTx at beginning of each iteration to save computing time
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)
        # Being iteration to solve for X based on fixed Y
        for u in range(num_user):
            conf_samp = conf[u,:].toarray() # Grab user row from confidence matrix and convert to dense
            pref = conf_samp.copy() 
            pref[pref != 0] = 1 # Create binarized preference vector 
            CuI = sparse.diags(conf_samp, [0]) # Get Cu - I term, don't need to subtract 1 since we never added it 
            yTCuIY = Y.T.dot(CuI).dot(Y) # This is the yT(Cu-I)Y term 
            yTCupu = Y.T.dot(CuI + Y_eye).dot(pref.T) # This is the yTCuPu term, where we add the eye back in
                                                      # Cu - I + I = Cu
            X[u] = spsolve(yTy + yTCuIY + lambda_eye, yTCupu) 
            # Solve for Xu = ((yTy + yT(Cu-I)Y + lambda*I)^-1)yTCuPu, equation 4 from the paper  
        # Begin iteration to solve for Y based on fixed X 
        for i in range(num_item):
            conf_samp = conf[:,i].T.toarray() # transpose to get it in row format and convert to dense
            pref = conf_samp.copy()
            pref[pref != 0] = 1 # Create binarized preference vector
            CiI = sparse.diags(conf_samp, [0]) # Get Ci - I term, don't need to subtract 1 since we never added it
            xTCiIX = X.T.dot(CiI).dot(X) # This is the xT(Cu-I)X term
            xTCiPi = X.T.dot(CiI + X_eye).dot(pref.T) # This is the xTCiPi term
            Y[i] = spsolve(xTx + xTCiIX + lambda_eye, xTCiPi)
            # Solve for Yi = ((xTx + xT(Cu-I)X) + lambda*I)^-1)xTCiPi, equation 5 from the paper
    # End iterations
    return X, Y.T # Transpose at the end to make up for not being transposed at the beginning. 
                         # Y needs to be rank x n. Keep these as separate matrices for scale reasons. 

In [23]:
user_vecs, item_vecs = implicit_weighted_ALS(product_train, lambda_val = 0.1, alpha = 15, iterations = 1,
                                            rank_size = 20)

### We can investigate ratings for a particular user by taking the dot product between the user and item vectors ($U$ and $V$). Let’s look at our first user.

In [24]:
user_vecs[0,:].dot(item_vecs).toarray()[0,:5]

array([5.24299219e-02, 9.85570586e-05, 1.25361284e-02, 3.89815739e-02,
       7.83943702e-03])

This is a sample of the first five items out of the 3664 in our stock. The first user in our matrix has the first item with the greatest recommendation out of the first five items. However, notice we only did one iteration because our algorithm was so slow! 

# Speeding Up ALS

In [25]:
#First, import the library so we can utilize it for our matrix factorization.
import implicit

In [26]:
alpha = 15
user_vecs, item_vecs = implicit.alternating_least_squares((product_train*alpha).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

This method is deprecated. Please use the AlternatingLeastSquares class instead


  0%|          | 0/50 [00:00<?, ?it/s]

## Evaluating the Recommender System

In [27]:
from sklearn import metrics

In [28]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)   
#Now, utilize this helper function inside of a second function that will calculate the AUC for each user in our training set that had at least one item masked. It should also calculate AUC for the most popular items for our users to compare.

def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popul

In [29]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

(0.772, 0.836)

We now have our recommender system trained and have proven it beats the benchmark of popularity. An AUC of 0.77 means the system is recommending items the user in fact had purchased in the test set far more frequently than items the user never ended up purchasing. To see an example of how it works, let’s examine the recommendations given to a particular user and decide subjectively if they make any sense.

First, however, we need to find a way of retrieving the items already purchased by a user in the training set. Initially, we will create an array of our customers and items we made earlier.

In [30]:
customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

Now, we can create a function that will return a list of the item descriptions from our earlier created item lookup table.

In [31]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    customer_id - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    customers_list - The array of customers used in the ratings matrix
    
    products_list - The array of products used in the ratings matrix
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    returns:
    
    A list of item IDs and item descriptions for a particular customer that were already purchased in the training set
    '''
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    return item_lookup.loc[item_lookup.Product_ID.isin(prod_codes)]

In [32]:
# We need to look these up by a customer’s ID. Looking at the list of customers:
customers_arr[:10]

array([11000, 11001, 11002, 11003, 11004, 11005, 11006, 11007, 11008,
       11009], dtype=int64)

### we can see that the first customer listed has an ID of 11000. Let’s examine their purchases from the training set.

In [33]:
get_items_purchased(11000, product_train, customers_arr, products_arr, item_lookup)

,Product_ID,Item_Description
5432,353,"Mountain-200 Silver, 38"
5442,214,"Sport-100 Helmet, Red"
5457,541,Touring Tire
5458,530,Touring Tire Tube
5459,573,"Touring-1000 Blue, 46"
5476,485,Fender Set - Mountain


We can see that the customer purchased a Mountain-200 Silver, 38,Sport-100 Helmet, Red,Touring Tire,Touring Tire Tube,Touring-1000 Blue, 46,Fender Set - Mountain. What items does the recommender system say this customer should purchase? We need to create another function that does this. Let’s also import the MinMaxScaler from scikit-learn to help with this.

from sklearn.preprocessing import MinMaxScaler

In [34]:
from sklearn.preprocessing import MinMaxScaler

In [35]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    '''
    This function will return the top recommended items to our users 
    
    parameters:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    num_items - The number of items you want to recommend in order of best recommendations. Default is 10. 
    
    returns:
    
    - The top n recommendations chosen based on the user/item vectors for items never interacted with/purchased
    '''
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code, item_lookup.Item_Description.loc[item_lookup.Product_ID == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'Product_ID': codes, 'Item_Description': descriptions}) # Create a dataframe 
    return final_frame[['Product_ID', 'Item_Description']] # Switch order of columns around

Essentially, this will retrieve the N highest ranking dot products between our user and item vectors for a particular user. Items already purchased are not recommended to the user. For now, let’s use a default of 10 items and see what the recommender system decides to pick for our customer.

In [36]:
rec_items(11000, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

,Product_ID,Item_Description
0,358,"Mountain-200 Black, 38"
1,362,"Mountain-200 Black, 46"
2,361,"Mountain-200 Black, 42"
3,478,Mountain Bottle Cage
4,352,"Mountain-200 Silver, 38"
5,563,"Touring-1000 Yellow, 54"
6,487,Hydration Pack - 70 oz.
7,562,"Touring-1000 Yellow, 50"
8,576,"Touring-1000 Blue, 60"
9,465,"Half-Finger Gloves, M"


These recommendations seem quite good! Remember that the recommendation system has no real understanding of what a ceramic jar is. All it knows is the purchase history. It identified that people purchasing a Mountain-200 Silver, 38,Sport-100 Helmet, Red,Touring Tire,Touring Tire Tube,Touring-1000 Blue, 46,Fender Set - Mountain may also want to purchase any of these items
    Mountain-200 Black, 38
	Mountain-200 Black, 46
	Mountain-200 Black, 38
	All-Purpose Bike Stand
    Mountain Bottle Cage
    Mountain-200 Silver, 38
    Mountain-200 Black, 42
	Mountain-200 Black, 46
	Touring-1000 Yellow, 60
	Sport-100 Helmet, Black. 
The recommender system also suggests jar magnets and a sugar dispenser, which is similar in use to a storage jar. I personally was blown away by how well the system seems to pick up on these sorts of shopping patterns. Let’s try another user that hasn’t made a large number of purchases.



In [37]:
get_items_purchased(11009, product_train, customers_arr, products_arr, item_lookup)

,Product_ID,Item_Description
5431,217,"Sport-100 Helmet, Black"
5585,359,"Mountain-200 Black, 38"
5661,575,"Touring-1000 Blue, 54"


This person seems like  want to put on Helemet while driving. What kind of items does the recommender system think she would be interested in?

In [38]:
rec_items(11009, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

,Product_ID,Item_Description
0,491,"Short-Sleeve Classic Jersey, XL"
1,357,"Mountain-200 Silver, 46"
2,561,"Touring-1000 Yellow, 46"
3,537,HL Mountain Tire
4,368,"Road-250 Red, 44"
5,463,"Half-Finger Gloves, S"
6,578,"Touring-2000 Blue, 50"
7,382,"Road-550-W Yellow, 38"
8,486,All-Purpose Bike Stand
9,574,"Touring-1000 Blue, 50"


It certainly picked up on 
    Mountain-200 Silver, 38
    Short-Sleeve Classic Jersey, XL
    Touring-1000 Yellow, 46
	HL Mountain Tire
	Half-Finger Gloves, L
	Mountain-200 Black, 46
	Touring-1000 Blue, 46
	Road-150 Red, 62
	Touring-1000 Yellow, 54
	Road-550-W Yellow, 38
Again, these recommendations seem very impressive given the system doesn’t understand the content behind the recommendations. Let’s try one more.



In [39]:
get_items_purchased(11008, product_train, customers_arr, products_arr, item_lookup)

,Product_ID,Item_Description
129,348,"Mountain-100 Black, 38"
5433,478,Mountain Bottle Cage
5434,477,Water Bottle - 30 oz.
5442,214,"Sport-100 Helmet, Red"
5513,361,"Mountain-200 Black, 42"
5829,561,"Touring-1000 Yellow, 46"


This customer seems like they are buying products suitable for lunch time. What other items does the recommender system think they might like?

In [40]:
rec_items(11008, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

,Product_ID,Item_Description
0,479,Road Bottle Cage
1,310,"Road-150 Red, 62"
2,573,"Touring-1000 Blue, 46"
3,575,"Touring-1000 Blue, 54"
4,217,"Sport-100 Helmet, Black"
5,541,Touring Tire
6,357,"Mountain-200 Silver, 46"
7,359,"Mountain-200 Black, 38"
8,530,Touring Tire Tube
9,360,"Mountain-200 Black, 42"


Once again, the recommender system comes through! Definitely a lot of bags and lunch related items in this recommendation list

### Commercial Impact of the recommendation system

we are going to take into consideration,the items recommended to customer's 11000 by our system and see the revenue and profit that would be generated when the customer enventually purchased those items.

In [49]:
retail_data.head()

,Product_ID,Item_Description,Customer_ID,OrderQuantity,Revenue,Cost
0,310,"Road-150 Red, 62",21768,1,3578.2700,2171.2942
1,346,"Mountain-100 Silver, 44",28389,1,3399.9900,1912.1544
2,346,"Mountain-100 Silver, 44",25863,1,3399.9900,1912.1544
3,336,"Road-650 Black, 62",14501,1,699.0982,413.1463
4,346,"Mountain-100 Silver, 44",11003,1,3399.9900,1912.1544


In [50]:
# recommended items for customer with id 11000 
rec_items = ['Mountain-200 Black, 38',
            'Mountain-200 Black, 46',
            'Mountain-200 Black, 38',
            'All-Purpose Bike Stand',
            'Mountain Bottle Cage',
            'Mountain-200 Silver, 38',
            'Mountain-200 Black, 42',
            'Mountain-200 Black, 46',
            'Touring-1000 Yellow, 60',
            'Sport-100 Helmet, Black']

In [76]:
#assume that our customer purchase these 4 items out of the 10 recommended products
impact=retail_data[retail_data['Item_Description'].isin([
            'All-Purpose Bike Stand',
            'Mountain Bottle Cage',
            'Mountain-200 Black, 38',
            'Sport-100 Helmet, Black'])]

In [77]:
impact.head()

,Product_ID,Item_Description,Customer_ID,OrderQuantity,Revenue,Cost
2212,358,"Mountain-200 Black, 38",12355,1,2049.0982,1105.81
2225,358,"Mountain-200 Black, 38",20208,1,2049.0982,1105.81
2258,358,"Mountain-200 Black, 38",12354,1,2049.0982,1105.81
2272,358,"Mountain-200 Black, 38",20206,1,2049.0982,1105.81
2318,358,"Mountain-200 Black, 38",12342,1,2049.0982,1105.81


In [78]:
# create a new column profit
impact['Profit'] = impact['Revenue']-impact['Cost']
impact.head()

<ipython-input-78-e364602bb2c1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  impact['Profit'] = impact['Revenue']-impact['Cost']


,Product_ID,Item_Description,Customer_ID,OrderQuantity,Revenue,Cost,Profit
2212,358,"Mountain-200 Black, 38",12355,1,2049.0982,1105.81,943.2882
2225,358,"Mountain-200 Black, 38",20208,1,2049.0982,1105.81,943.2882
2258,358,"Mountain-200 Black, 38",12354,1,2049.0982,1105.81,943.2882
2272,358,"Mountain-200 Black, 38",20206,1,2049.0982,1105.81,943.2882
2318,358,"Mountain-200 Black, 38",12342,1,2049.0982,1105.81,943.2882


In [79]:
# let sum up the data per transaction
comercial_impact= impact.groupby(["Item_Description"]).agg({'OrderQuantity':sum, 'Revenue':sum,'Profit':sum}).reset_index()
comercial_impact.head()

,Item_Description,OrderQuantity,Revenue,Profit
0,All-Purpose Bike Stand,249,3.959100e+04,24783.9660
1,Mountain Bottle Cage,2025,2.022975e+04,12663.7425
2,"Mountain-200 Black, 38",582,1.294866e+06,590477.4604
3,"Sport-100 Helmet, Black",2085,7.295415e+04,45669.2145


In [ ]:
comercial_impact[]